In [1]:
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [2]:
df = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df['stimulant'] = df['stimulant'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')
df


,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,3,1,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,low,1.0,0.000000
2,2,4,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,low,1.0,2.000000
3,3,1,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,None,0.0,0.333333
4,1,3,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,low,1.5,0.000000
5,3,4,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,low,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1,0,25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,None,0.0,1.000000
1885,1,0,33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,None,0.0,3.666667
1886,2,2,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,high,5.0,2.666667
1887,1,0,45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,None,0.0,3.000000


In [3]:
stimtestdata = df.drop(['hallucinagen','depressant'],axis = 1)
stimx = stimtestdata.drop('stimulant',axis = 1)
stimy = stimtestdata.stimulant



In [4]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)

In [5]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

500      low
864     high
186      low
865     None
673     None
        ... 
547     None
99       low
1101    None
1835     low
580      low
Name: stimulant, Length: 1413, dtype: object

Logistic regression classifiers

In [46]:
logregparam_grid = {'C': [0.001,0.1,1, 10],'max_iter' :[1000],'class_weight':['balanced'],'warm_start' :[True,False], 'solver': ['liblinear', 'rbf','sag']}
mod = GridSearchCV(LogisticRegression(), logregparam_grid, n_jobs=-1, cv=5, verbose=1)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished


In [56]:
standardmodel1 = mod.best_estimator_
standardmodel1

LogisticRegression(C=1, class_weight='balanced', max_iter=1000,
                   solver='liblinear', warm_start=True)

In [48]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.5888986444814821, 0.6327670094613298)

In [49]:
logregparam_grid2 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'C': [0,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[1000],'class_weight':[None,'balanced']
                     ,'warm_start' :[True,False]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga',]
                     ,'dual':[False,True],"l1_ratio":[None,0.01,0.1,0.2,0.3,0.4]
                    ,'fit_intercept':[True,False]}
mod2 = GridSearchCV(LogisticRegression(), logregparam_grid2, n_jobs=-1, cv=5, verbose=1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

Fitting 5 folds for each of 51840 candidates, totalling 259200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 3560 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 19560 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 41960 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 65400 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 91856 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 116456 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 140636 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 162260 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 189866 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 213096 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 241454 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 259200 out of 259200 | elapsed:  5.3min finished


In [57]:
standardmodel2 = mod2.best_estimator_
standardmodel2

LogisticRegression(C=1, max_iter=1000, solver='sag', tol=0.1)

In [51]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')

(0.5938632715685083, 0.6257621507389974)

In [52]:
logregparam_grid3 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'C': [0,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[1000],'class_weight':[None]
                     ,'warm_start' :[True,False]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga',]
                     ,'dual':[False,True],"l1_ratio":[None,0.01,0.1,0.2,0.3,0.4]
                    ,'fit_intercept':[True,False]}
mod3 = GridSearchCV(LogisticRegression(), logregparam_grid3, n_jobs=-1, cv=5, verbose=0)
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "


In [58]:
standardmodel3 = mod3.best_estimator_
standardmodel3

LogisticRegression(C=0.001, l1_ratio=0.3, max_iter=1000, solver='sag', tol=10,
                   warm_start=True)

In [73]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')


(0.5711277961755238, 0.5724005867014341)

In [77]:
classification_report(stimYtrain,stimtrianlog3ypred)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n        None       0.74      0.62      0.68       597\n        high       0.00      0.00      0.00       198\n         low       0.55      0.80      0.65       618\n\n    accuracy                           0.62      1413\n   macro avg       0.43      0.48      0.44      1413\nweighted avg       0.55      0.62      0.57      1413\n'

In [60]:
pkl.dump(standardmodel1,open('StandardModel1.pkl','wb'))
pkl.dump(standardmodel2,open('StandardModel2.pkl','wb'))
pkl.dump(standardmodel3,open('StandardModel3.pkl','wb'))

Decision Trees

In [85]:
parameters={'min_samples_split' : range(10,500,20),'max_depth': range(1,20,1),'ccp_alpha':[x/60 for x in list(range(0,60,1))],'max_features':range(1,30,1),'class_weight':['balanced'],'criterion':['gini']}
dtc = GridSearchCV(DecisionTreeClassifier(DecisionTreeClassifier()),param_grid=parameters,verbose=True,n_jobs=-1)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass criterion=DecisionTreeClassifier() as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Fitting 5 folds for each of 826500 candidates, totalling 4132500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1420 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 4452 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 8652 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 19116 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 32316 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 47916 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-1)]: Done 65916 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 86316 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 109116 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 134316 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 161916 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 191916 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 224316 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done

In [101]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.5344461165030636, 0.4984071172586556)

In [87]:
parameters={'min_samples_split' : range(10,500,20),'ccp_alpha':[x/200 for x in list(range(0,100,1))],'max_features':range(1,30,1),'class_weight':['balanced'],'criterion':['entropy','gini']}
dtc2 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc2.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc2.predict(stimXtrain)
dtcytestpred = dtc2.predict(stimXtest)

Fitting 5 folds for each of 1653000 candidates, totalling 8265000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 2600 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 10600 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 21800 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 36200 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 53800 tasks      | elapsed:   46.7s
[Parallel(n_jobs=-1)]: Done 74600 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 98600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 125800 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 156200 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 189800 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 226600 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 266600 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 309800 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: D

In [100]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')


(0.5344461165030636, 0.4984071172586556)

In [102]:
parameters={'min_impurity_decrease':[0+x/100 for x in range(1,100)],'ccp_alpha':[x/100 for x in range(0,100,1)],'class_weight':['balanced'],'criterion':['entropy','gini']}
dtc3 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc3.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc3.predict(stimXtrain)
dtcytestpred = dtc3.predict(stimXtest)

Fitting 5 folds for each of 19800 candidates, totalling 99000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1420 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 17292 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 39692 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 68492 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 98736 tasks      | elapsed:   46.8s
[Parallel(n_jobs=-1)]: Done 99000 out of 99000 | elapsed:   46.9s finished


In [103]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')


(0.5344461165030636, 0.4984071172586556)

In [104]:
standardmodel4 = dtc.best_estimator_
standardmodel5 = dtc2.best_estimator_
standardmodel6 = dtc3.best_estimator_

pkl.dump(standardmodel4,open('StandardModel4.pkl','wb'))
pkl.dump(standardmodel5,open('StandardModel5.pkl','wb'))
pkl.dump(standardmodel6,open('StandardModel6.pkl','wb'))




Knearest

In [105]:
# knn = KNeighborsClassifier(n_neighbors=43,weights='uniform')
# knn.fit(stimXtrain,stimYtrain)
# knntrainypred=knn.predict(stimXtrain)
# knntestypred=knn.predict(stimXtest)
# f1_score(stimYtrain,knntrainypred,average='weighted'),f1_score(stimYtest,knntestypred,average='weighted')
parameters={'n_neighbors':range(1,50)}
knn = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn.fit(stimXtrain,stimYtrain)
knnytrainpred = knn.predict(stimXtrain)
knnytestpred = knn.predict(stimXtest)



Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    0.6s finished


In [107]:
parameters={'n_neighbors':range(50,1130)}
knn2 = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn2.fit(stimXtrain,stimYtrain)
knn2ytrainpred = knn2.predict(stimXtrain)
knn2ytestpred = knn2.predict(stimXtest)


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 748 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 2748 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed:   23.2s finished


In [109]:
standardmodel7 = knn.best_estimator_
standardmodel8 = knn2.best_estimator_

pkl.dump(standardmodel7,open('StandardModel7.pkl','wb'))
pkl.dump(standardmodel8,open('StandardModel8.pkl','wb'))

Random Forest

In [113]:
# rfc = RandomForestClassifier(criterion='gini',ccp_alpha=0.005,n_estimators=1000,max_features=6,class_weight='balanced',max_depth=6,oob_score=True)
# rfc.fit(stimXtrain,stimYtrain)
parameters={'ccp_alpha':[x/10 for x in list(range(0,10,1))],'max_features':range(1,30,1),'class_weight':['balanced'],'criterion':['entropy'],'n_estimators':[100,200,300,400,500,1000,2000]}
rfc = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc.fit(stimXtrain,stimYtrain)
rfcytrainpred = rfc.predict(stimXtrain)
rfcytestpred = rfc.predict(stimXtest)



Fitting 5 folds for each of 2030 candidates, totalling 10150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 10150 out of 1015

In [ ]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')

In [115]:
parameters={'ccp_alpha':[x/10 for x in list(range(0,10,1))]
                         ,'random_state':[True],'max_features':[0,1,2,3,4,5,6,7,15,30,43]
                         ,'class_weight':['balanced'],'criterion':['gini']
                         ,'oob_score':[True,False],'n_estimators':[100,200,300,400,500,1000,2000]}
rfc2 = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc2.fit(stimXtrain,stimYtrain)
rfc2ytrainpred = rfc2.predict(stimXtrain)
rfc2ytestpred = rfc2.predict(stimXtest)



Fitting 5 folds for each of 1540 candidates, totalling 7700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 7700 out of 7700 | elapsed: 24.4min finished


In [116]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')

(1.0, 0.6652202981862024)

In [118]:
standardmodel9 = rfc.best_estimator_
standardmodel10 = rfc2.best_estimator_

pkl.dump(standardmodel9,open('StandardModel9.pkl','wb'))
pkl.dump(standardmodel10,open('StandardModel10.pkl','wb'))

voting classifier

In [24]:
vtc = VotingClassifier([('dtc2',dtc2),('rfc2',rfc2),('knn',knn),('knn2',knn2),('log',mod),('log3',mod3),('log2',mod2),('dtc',dtc),('dtc3',dtc3),("rfc",rfc)],voting='hard',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('dtc2',
                              DecisionTreeClassifier(ccp_alpha=0.0052,
                                                     criterion='entropy',
                                                     max_depth=50)),
                             ('rfc2',
                              RandomForestClassifier(ccp_alpha=0.01,
                                                     class_weight='balanced',
                                                     criterion='entropy',
                                                     n_estimators=10000,
                                                     oob_score=True)),
                             ('knn', KNeighborsClassifier(n_neighbors=43)),
                             ('knn2', KNeighborsClassifier(n_neighbors=11)),
                             ('log',
                              LogisticRegres...
                              LogisticRegression(class_weight='balanced',
                                 

In [25]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.6260190582728704 0.6084518184576705


In [26]:
pkl.dump(vtc,open('vtc.pkl','wb'))